<a href="https://colab.research.google.com/github/fovi-llc/aider-swe-bench/blob/main/notebooks/aider_swe_bench_colab_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Have to use an older version of Aider because something has changed in `Coder.create` because we get this error with current versions:

```
TypeError: Coder.init() got an unexpected keyword argument 'git_dname'
```

Same thing will happen with `max_chat_history_tokens` if `git_dname` is removed from the args to the `get_coder` call in `process_one_instance` (`harness.py` line 231).

I picked ~~0.40.6 because last version from the time `aider-swe-bench` was run before a major version bump~~ 0.46.1 because latest that doesn't fail.  Actually 0.46.1 and 0.47.1 hang on user input until ctrl-C (Coder.keyboard_interrupt).
0.48.1 appears to be first that fails for `git_name`.  Back to 0.41.0 fail by hanging.


# Install packages for this notebook.

* Do this once then restart session.

In [1]:
%%bash

cd /content

if [[ ! -d aider-swe-bench ]]; then
  # Clone this repo
  git clone https://github.com/fovi-llc/aider-swe-bench

  # Clone the SWE Bench docker repo into a subdir of this repo
  cd aider-swe-bench
  git clone https://github.com/aorwall/SWE-bench-docker

  # Install pip requirements
  pip install -qUr requirements.txt

  # You may want to install the latest main branch of aider
  # python -m pip install --upgrade git+https://github.com/paul-gauthier/aider.git
  # python -m pip install "aider-chat==0.40.6"
  # pip install -U git+https://github.com/fovi-llc/aider.git

  # pip install -U lox swebench==1.1.5 pre-commit git+https://github.com/fovi-llc/aider.git

  # Need latest datasets because of fsspec issue
  pip install -qU datasets huggingface_hub

fi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.8/464.8 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Cloning into 'aider-swe-bench'...
Cloning into 'SWE-bench-docker'...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.30.0 which is incompatible.
google-colab 1.0.0 requires tornado==6.4.2, but you have tornado 6.4.1 which is incompatible.
flask 3.1.0 requires blinker>=1.9, but you have blinker 1.8.2 which is incompatible.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.4.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have 

## Restart session after running this because the version of the `huggingface_hub` is updated.

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !sudo ln -s /usr/local/bin/udocker /usr/local/bin/docker
        !useradd -m user > /dev/null
        !chsh -s /usr/bin/bash user
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


In [2]:
!docker

Error: do not run as root !


In [3]:
!su - user -c "docker images"

Info: creating repo: /home/user/.udocker
Info: udocker command line interface 1.3.17
Info: searching for udockertools >= 1.2.11
Info: installing udockertools 1.2.11
Info: installation of udockertools successful
REPOSITORY


## Connect Google Drive for model cache

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install Ollama

Need `lshw` command for Ollama install script to detect GPU.

In [4]:
!apt-get install -y lshw
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  pci.ids usb.ids
The following NEW packages will be installed:
  lshw pci.ids usb.ids
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 790 kB of archives.
After this operation, 2,988 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 790 kB in 2s (382 kB/s)
Selecting previously unselected package lshw.
(Reading database ... 124950 files and directories currently installed.)
Preparing to unpack .../lshw_02.19.git.2021.06.19.996aaad9c7-2build1_amd64.deb ...
Unpacking lshw (02.19.git.2021.06.19.996aaad9c7-2build1) ...
Se

In [5]:
with open(f'/home/user/.aider.model.settings.yml', 'w') as f_out:
    f_out.write("""
- name: ollama_chat/deepseek-r1:14b
  # edit_format: diff
  edit_format: whole # diff wasn't working
  weak_model_name: null
  use_repo_map: true
  send_undo_reply: false
  lazy: false
  reminder: sys
  examples_as_sys_msg: true
  cache_control: false
  caches_by_default: true
  use_system_prompt: true
  use_temperature: true
  streaming: true
  extra_params:
    num_ctx: 8192 # How large a context window?
- name: ollama_chat/qwen2.5-coder:14b
  extra_params:
    num_ctx: 16384
- name: ollama_chat/mistral-small
  extra_params:
    num_ctx: 16384
""")

### Configure working directories and set permissions for the `user` user.

In [6]:
%cd /content/aider-swe-bench
!mkdir -p /tmp/aider
!mkdir -p /opt/logs
!mkdir -p chat-logs
!mkdir -p repos
!chown -R user:user /tmp/aider /opt/logs chat-logs repos predictions /home/user
# !chmod -R go+rX repos

/content/aider-swe-bench


### Fetch dataset from HF.  Do this as root before we run the harness which runs as user and can't update the ~root/.cache.

In [7]:
# from google.colab import userdata
# os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

from utils import get_lite_dataset
from harness import process_one_instance, REPOS_DNAME, CHAT_LOGS_DNAME, PREDS_DNAME

print(f"{REPOS_DNAME=}")
print(f"{CHAT_LOGS_DNAME=}")
print(f"{PREDS_DNAME=}")

dataset = get_lite_dataset()
print(f"{len(dataset)=}")

REPOS_DNAME=PosixPath('repos')
CHAT_LOGS_DNAME=PosixPath('chat-logs')
PREDS_DNAME=PosixPath('predictions')
dataset: princeton-nlp/SWE-bench_Lite


README.md:   0%|          | 0.00/3.67k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/120k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

Generating dev split:   0%|          | 0/23 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

len(dataset)=300


# The cells above should only be run once per session invocation.

If you restart the session then you need to start with the next cell (i.e. start the Ollama server).

### Start Ollama server

In [11]:
!nohup ollama serve > ollama.log 2>&1 &

In [9]:
import os

os.environ["OLLAMA_MODELS"] = "/content/drive/MyDrive/Data/Ollama"
!mkdir -p $OLLAMA_MODELS

os.environ["OLLAMA_API_BASE"] = "http://127.0.0.1:11434"

# Step 1: Set the environment variable
# os.environ["OLLAMA_MODEL"] = "deepseek-r1:14b"
# os.environ["OLLAMA_MODEL"] = "deepseek-coder:6.7b"
# os.environ["OLLAMA_MODEL"] = "qwen2.5-coder:14b"
os.environ["OLLAMA_MODEL"] = "mistral-small"

# Step 2: Retrieve the environment variable
model_name = os.getenv("OLLAMA_MODEL")

model_name

'mistral-small'

In [13]:
%%capture out

!ollama pull $model_name

#TODO: Check if pull successful

In [14]:
out.show()

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 102a747c1376... 100% ▕▏  14 GB                         
pulling 6db27cd4e277... 100% ▕▏  695 B                         
pulling 6d7b25ffd247... 100% ▕▏  644 B                         
pulling 43070e2d4e53... 100% ▕▏  11 KB                         
pulling e0daf17ff83e... 100% ▕▏   21 B                         
pulling 41ffc852c4b6... 100% ▕▏  562 B                         
verifying sha256 digest 
writing manifest 
success 


## One one or more specific instances.

See below for using regex to match multiple instances.

In [ ]:
%%capture out

%cd /content/aider-swe-bench

aider_model = f"ollama_chat/{model_name}"
command = f"python harness.py --models {aider_model} --num_tries 1 -i sympy__sympy-18532"
!su user -w OLLAMA_API_BASE --command "{command}"

In [ ]:
out.show()

/content/aider-swe-bench
models:
[
    "ollama_chat/mistral-small"
]
temperature: 0
out_dname: predictions/terse-udiff--ollama_chat-mistral-small
len(done_instances): 0
prior_dnames: []
len(prior_preds): 0
len(plausible_instances): 0
sorted(remaining_instances):
[
    "sympy__sympy-18532"
]
len(remaining_instances): 1

instance_id: sympy__sympy-18532
expr.atoms() should return objects with no args instead of subclasses of Atom
`expr.atoms()` with no arguments returns subclasses of `Atom` in `expr`. But the correct definition of a leaf node should be that it has no `.args`. 

This should be easy to fix, but one needs to check that this doesn't affect the performance. 


attempt, model: 1, ollama_chat/mistral-small
git_tempdir: /tmp/aider/tmpizjyuqi7
https://github.com/sympy/sympy 74227f900b05009d4eed62e34a166228788a32ca
Cloning into '/tmp/aider/tmpizjyuqi7'...
done.
HEAD is now at 74227f900b Merge pull request #18518 from mijo2/7763_pip_installation
git_dname: /tmp/aider/tmpizjyuqi7
Aid

## Run for all instances that start with "psf"

In [ ]:
%%capture out

%cd /content/aider-swe-bench

aider_model = f"ollama_chat/{model_name}"
command = f"python harness.py --models {aider_model} --num_tries 1 -f ^psf"
!su user -w OLLAMA_API_BASE --command "{command}"

In [ ]:
import datetime
now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
with open(f'predictions/stdout_{now}.txt', 'w') as f_out:
    f_out.write(out.stdout)

!mkdir -p /content/drive/MyDrive/Data/aider_swe_bench_
!rsync -aR predictions /content/drive/MyDrive/Data/aider_swe_bench

out.show()

# Run the whole test suite - lite to start with of course

# Copy predictions that are already done.

In [17]:
!rsync -arR /content/drive/MyDrive/Data/aider_swe_bench/predictions/terse-udiff--ollama_chat-{model_name} predictions

In [12]:
%%capture out

%cd /content/aider-swe-bench

aider_model = f"ollama_chat/{model_name}"
command = f"python harness.py --models {aider_model} --num_tries 1"
!su user -w OLLAMA_API_BASE --command "{command}"

In [13]:
import datetime
now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
with open(f'predictions/stdout_{now}.txt', 'w') as f_out:
    f_out.write(out.stdout)

!mkdir -p /content/drive/MyDrive/Data/aider_swe_bench
!rsync -aR predictions /content/drive/MyDrive/Data/aider_swe_bench

out.show()

/content/aider-swe-bench
models:
[
    "ollama_chat/mistral-small"
]
temperature: 0
out_dname: predictions/terse-udiff--ollama_chat-mistral-small
len(done_instances): 102
prior_dnames: []
len(prior_preds): 0
len(plausible_instances): 0
sorted(remaining_instances):
[
    "astropy__astropy-12907",
    "astropy__astropy-14995",
    "astropy__astropy-6938",
    "astropy__astropy-7746",
    "django__django-11039",
    "django__django-11049",
    "django__django-11099",
    "django__django-11179",
    "django__django-11422",
    "django__django-11742",
    "django__django-11797",
    "django__django-11815",
    "django__django-11848",
    "django__django-11905",
    "django__django-11999",
    "django__django-12113",
    "django__django-12125",
    "django__django-12308",
    "django__django-12470",
    "django__django-12589",
    "django__django-12708",
    "django__django-12747",
    "django__django-12983",
    "django__django-13033",
    "django__django-13158",
    "django__django-13220",